### Measurement of observables includes that the hamiltonian is split into pauli strings and measured. 
#### The following notebook contains various examples of the same. 

In [34]:
from qiskit_ibm_runtime import QiskitRuntimeService, Session, Sampler, Estimator, Options
from qiskit.test.reference_circuits import ReferenceCircuits
from qiskit.quantum_info import *
from qiskit import *
import numpy as np 

# Initialize account.
service = QiskitRuntimeService()

# Set options, which can be overwritten at job level.
options = Options(optimization_level=3, resilience_level=2)

### Example 1

Hadamard gate as the observable.

$ \langle \psi | H | \psi \rangle $

$ | \psi \rangle = |1 \rangle $

$ H |1 \rangle = \frac{1}{\sqrt2} (|0 \rangle - |1 \rangle) $

$  \langle 1 | H | 1 \rangle = \langle 1 |  \frac{1}{\sqrt2} (|0 \rangle - |1 \rangle) = \frac{1}{\sqrt2} ( \langle1|0\rangle - \langle1|1\rangle ) = -\frac{1}{\sqrt2} = -0.70710678 $

In [37]:
psi = QuantumCircuit(1)
psi.x(0)

H1 = SparsePauliOp.from_operator([
    [1/np.sqrt(2), 1/np.sqrt(2)],
    [1/np.sqrt(2), -1/np.sqrt(2)]
])

print (H1)

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session=session, options=options)
    job = estimator.run(
        circuits=[psi], observables=[H1]
    )
    print(f"Estimator results: {job.result()}")
    session.close()

SparsePauliOp(['X', 'Z'],
              coeffs=[0.70710678+0.j, 0.70710678+0.j])
Estimator results: EstimatorResult(values=array([-0.73014668]), metadata=[{'zne': {'noise_amplification': {'noise_amplifier': "<TwoQubitAmplifier:{'noise_factor_relative_tolerance': 0.01, 'random_seed': None, 'sub_folding_option': 'from_first'}>", 'noise_factors': [1, 3, 5], 'values': [-0.7290270914033304, -0.7032176938900214, -0.7056925676241743], 'variance': [0.4995194999999999, 0.4999848749999999, 0.4999979999999999], 'shots': [4000, 4000, 4000]}, 'extrapolation': {'extrapolator': 'LinearExtrapolator'}}}])


#### Example 2

$\langle \psi | ZZ | \psi\rangle $

$ \quad = \frac{1}{\sqrt2}.\frac{1}{\sqrt2} \begin{bmatrix} \langle 00| + \langle 11 | ZZ | 00 \rangle + |11\rangle \end{bmatrix} $

$ \quad = \frac{1}{2} \begin{bmatrix}\begin{bmatrix} \langle 00| + \langle 11 \end{bmatrix} |  \begin{bmatrix} Z|0 \rangle.Z|0 \rangle + Z|1 \rangle.Z|1\rangle\end{bmatrix} \end{bmatrix} $

$ \quad = \frac{1}{2} \begin{bmatrix}\begin{bmatrix} \langle 00| + \langle 11 \end{bmatrix} |  \begin{bmatrix} |00 \rangle + |11 \rangle \end{bmatrix} \end{bmatrix} $

$ \quad = \frac{1}{2}.2 = 1 $


In [40]:
psi = QuantumCircuit(1)
psi.x(0)

Z = [
    [1,0],
    [0,-1]
]

ZZ = np.kron(Z,Z)

H1 = SparsePauliOp.from_operator(ZZ)

print (H1)

psi = QuantumCircuit(2)
psi.h(0)
psi.cx(0,1)

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session=session, options=options)
    job = estimator.run(
        circuits=[psi], observables=[H1]
    )
    print(f"Estimator results: {job.result()}")
    session.close()


SparsePauliOp(['ZZ'],
              coeffs=[1.+0.j])
Estimator results: EstimatorResult(values=array([1.]), metadata=[{'zne': {'noise_amplification': {'noise_amplifier': "<TwoQubitAmplifier:{'noise_factor_relative_tolerance': 0.01, 'random_seed': None, 'sub_folding_option': 'from_first'}>", 'noise_factors': [1, 3, 5], 'values': [1.0, 1.0, 1.0], 'variance': [0.0, 0.0, 0.0], 'shots': [4000, 4000, 4000]}, 'extrapolation': {'extrapolator': 'LinearExtrapolator'}}}])


#### Example 3

$\langle \psi | ZZ + XX | \psi\rangle $

$ \quad = \frac{1}{\sqrt2}.\frac{1}{\sqrt2} \begin{bmatrix} \langle 00| + \langle 11 | ZZ + XX | 00 \rangle + |11\rangle \end{bmatrix} $

$ \quad = \frac{1}{2} \begin{bmatrix} \begin{bmatrix} \langle 00| + \langle 11 \end{bmatrix} |  \begin{bmatrix} \begin{bmatrix} Z|0 \rangle.Z|0 \rangle + Z|1 \rangle.Z|1\rangle \end{bmatrix} \end{bmatrix} + \begin{bmatrix} X|0 \rangle.X|0 \rangle + X|1 \rangle.X|1\rangle \end{bmatrix} \end{bmatrix}  $

$ \quad = \frac{1}{2} \begin{bmatrix}\begin{bmatrix} \langle 00| + \langle 11 \end{bmatrix} |  \begin{bmatrix} \begin{bmatrix} |00 \rangle + |11 \rangle \end{bmatrix} \end{bmatrix} + \begin{bmatrix} |11 \rangle + |00 \rangle \end{bmatrix} \end{bmatrix} $

$ \quad = \frac{1}{2}.(2 + 2) = 2 $


In [41]:
psi = QuantumCircuit(1)
psi.x(0)

Z = [
    [1,0],
    [0,-1]
]

X = [
    [0,1],
    [1,0]
]

ZZ = np.kron(Z,Z)
XX = np.kron(X,X)

H1 = SparsePauliOp.from_operator(np.add(ZZ, XX))

print (H1)

psi = QuantumCircuit(2)
psi.h(0)
psi.cx(0,1)

with Session(service=service, backend="ibmq_qasm_simulator") as session:
    estimator = Estimator(session=session, options=options)
    job = estimator.run(
        circuits=[psi], observables=[H1]
    )
    print(f"Estimator results: {job.result()}")
    session.close()


SparsePauliOp(['XX', 'ZZ'],
              coeffs=[1.+0.j, 1.+0.j])
Estimator results: EstimatorResult(values=array([2.]), metadata=[{'zne': {'noise_amplification': {'noise_amplifier': "<TwoQubitAmplifier:{'noise_factor_relative_tolerance': 0.01, 'random_seed': None, 'sub_folding_option': 'from_first'}>", 'noise_factors': [1, 3, 5], 'values': [2.0, 2.0, 2.0], 'variance': [0.0, 0.0, 0.0], 'shots': [4000, 4000, 4000]}, 'extrapolation': {'extrapolator': 'LinearExtrapolator'}}}])
